In [14]:
import psycopg2
import os
from dotenv import load_dotenv
from psycopg2.extras import RealDictCursor
from datetime import datetime, timedelta
from app import convert_date_to_week_format, get_db_connection

# .env 파일에서 환경 변수 로드
load_dotenv()

True

<connection object at 0x0000021358EDE130; dsn: 'user=postgres.cctepydjljbpcvvydixp password=xxx dbname=postgres host=aws-1-ap-southeast-1.pooler.supabase.com port=6543 sslmode=require', closed: 0>

In [ ]:
supplier_filter = 'MOKA'
to_site_filter = 'EACM'
week_from = request.args.get('week_from', default_weekname)
week_to = request.args.get('week_to', default_weekname_to)